In [9]:
import pandas as pd
import plotly.express as px
import warnings 
import ipywidgets as widgets
from IPython.display import clear_output
import folium
import folium.plugins
warnings.filterwarnings('ignore')

from folium import IFrame

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df = pd.read_csv('can_scores_quintiles_EN.csv')
df = df.rename(columns={'PRCDDA': 'poi_cbg'})
df['Type'] = 'Urban'
df.loc[df['Dissemination area (DA) Population'] < 400, 'Type'] = 'Rural'
df[df['Dissemination area (DA) Population'] >= 400]
df = df[['poi_cbg', 'Dissemination area (DA) Population', 'Type']]
df

,poi_cbg,Dissemination area (DA) Population,Type
0,10010165,539,Urban
1,10010166,264,Rural
2,10010167,341,Rural
3,10010168,549,Urban
4,10010169,296,Rural
...,...,...,...
54791,59590023,440,Urban
54792,59590024,318,Rural
54793,59590025,683,Urban
54794,59590026,583,Urban


In [12]:
data = pd.read_csv('combined.csv')
# drop rows with Nan values in raw_visit_counts

data = data.dropna(subset=['raw_visit_counts'])
data['mobility_score'] = data['normalized_visits_by_state_scaling'] * data['median_dwell'] / 60

In [13]:
data = data.dropna(subset=['poi_cbg'])
data['poi_cbg'] = (data['poi_cbg'].str[3:]).astype(int)
df1 = data.merge(df, on='poi_cbg')

In [21]:
df1['date_range_start'] = df1['date_range_start'].str[:10]
df1 = df1.loc[df1['date_range_start'] >= '2019-06-01']
df1 = df1.loc[df1['date_range_start'] <= '2021-09-01']
df2 = df1.groupby(['date_range_start', 'region', 'Type']).sum()
df2.to_csv('dataframe.csv')
df2 = pd.read_csv('dataframe.csv')
df2

,date_range_start,region,Type,Unnamed: 0,naics_code,latitude,longitude,enclosed,phone_number,is_synthetic,wkt_area_sq_meters,raw_visit_counts,raw_visitor_counts,poi_cbg,distance_from_home,median_dwell,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,normalized_visits_by_total_visits,normalized_visits_by_total_visitors,mobility_score,Dissemination area (DA) Population
0,2019-06-03,AB,Rural,127623,79476294.0,5730.822100,-12514.344248,4,1.626486e+12,0,187250.0,841.0,621.0,5289321836,3661002.0,9417.5,1.154753e+05,0.028835,0.037668,0.001476,0.002073,1.093263e+05,32145
1,2019-06-03,AB,Urban,569083,365590860.0,26505.743255,-57546.841334,8,7.591217e+12,18,712695.0,3787.0,2854.0,24332374079,3312873.0,47203.5,5.200505e+05,0.130805,0.173914,0.006662,0.009551,7.400490e+05,770386
2,2019-06-03,BC,Rural,110831,68638611.0,4787.313436,-11626.441491,2,1.340820e+12,3,152734.0,660.0,496.0,5628313303,3360363.0,10049.0,8.667633e+04,0.014457,0.019787,0.000926,0.001375,1.272873e+05,29486
3,2019-06-03,BC,Urban,984382,612689786.0,41900.402501,-103942.077900,29,1.233708e+13,13,1284333.0,7522.0,5987.0,50184824354,14864359.0,76569.0,9.878475e+05,0.224333,0.291299,0.010549,0.016594,1.116625e+06,843318
4,2019-06-03,MB,Rural,16262,9392645.0,647.360375,-1267.710226,0,1.445603e+11,0,10390.0,59.0,39.0,599266247,10649.0,2017.5,9.422362e+03,0.008861,0.011978,0.000392,0.000491,3.177519e+04,4125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2353,2021-08-30,PE,Urban,25607,11560186.0,740.310145,-1009.632006,0,3.044296e+11,1,293331.0,149.0,130.0,176302260,357068.0,579.5,2.398090e+04,0.157766,0.200310,0.006009,0.014217,1.417394e+04,8326
2354,2021-08-30,QC,Rural,199643,137277194.0,8766.228173,-13857.195344,4,2.765284e+12,2,502589.0,2649.0,1909.0,4669995778,3957123.0,15532.5,3.530605e+05,0.050439,0.075701,0.001814,0.003552,3.639406e+05,64320
2355,2021-08-30,QC,Urban,1290269,843171015.0,53551.999875,-85370.426905,25,1.694033e+13,26,8250781.0,14212.0,10974.0,28691774190,27108656.0,86915.0,1.894185e+06,0.198558,0.274569,0.009731,0.020418,1.639705e+06,983964
2356,2021-08-30,SK,Rural,38338,22397859.0,1600.002718,-3272.862316,0,3.920041e+11,0,16274.0,311.0,240.0,1460070436,1641164.0,1691.0,2.835177e+04,0.036182,0.048340,0.001157,0.002445,2.584022e+04,9633


In [22]:
fig = px.bar(df2, x="region", y="mobility_score", animation_frame="date_range_start", range_y= [0,3_000_000] ,animation_group="region", color='Type')
fig.show()

In [25]:
fig = px.line(df2[df2['Type'] == 'Rural'], x='date_range_start', y='mobility_score', color='region')
fig.show()

In [26]:
fig = px.line(df2[df2['Type'] == 'Urban'], x='date_range_start', y='mobility_score', color='region')
fig.show()